# CADP tools queries 

## 1. MCL Queries

In [5]:
a = !ls /home/abdelouahab/PFE/lcm-tests/cadp/graph/bcg/*.bcg
e = [print(i) for i in a]

/home/abdelouahab/PFE/lcm-tests/cadp/graph/bcg/9M-5-[1-2001]-[a]-lcm.bcg
/home/abdelouahab/PFE/lcm-tests/cadp/graph/bcg/sample.bcg


In [12]:
# Choose target graph 
file_name = "sample.bcg"
file_path = f"../graph/bcg/{file_name}"

In [13]:
%%bash 
export PATH=$PATH:$CADP/bin.x64:$CADP/com

In [7]:
def print_query_content(query_path):
    !cat {query_path}

### 1.1  Product adoption query

In [8]:
# Query content 
query_name = "product_adoption.mcl"
query_path = f"../queries/mcl/{query_name}"
print_query_content(query_path)

(* Product Adoption :*)

macro QUERY () =
     {!"G"}.
     {SELF ?C0:String ?P0:String !"T0"}.
     {?any}*.
     {SELF ?C1:String ?P1:String !"T1" where
        (diff (string2natset (P1), string2natset (P0)) <> empty)
     }
end_macro

macro FINAL_STATE () =
    [ not { SELF ... } ] false
end_macro

<
    (true* . QUERY . true*)+
> FINAL_STATE


In [10]:
%%bash -s $file_path $query_path

$CADP/com/bcg_open $1 evaluator4 -diag $2

bcg_open: using ``/home/abdelouahab/cadp//bin.x64/evaluator4.a''
bcg_open: running ``evaluator4 -diag ../queries/mcl/product_adoption.mcl'' for ``../graph/bcg/sample.bcg''

TRUE
*** diagnostic sequence found at depth 5

<initial state>
"G"
"SELF !:1:2:3:4:5:6: !:1:3:4: !T0"
"G"
"SELF !:1:2:3:4:5:6:9: !:1:3:15 !T1"
"ST"
<goal state>


/tmp/caesar_19855_1.c: In function 'caesar_iterate_boolean_variable_0':
/tmp/caesar_19855_1.c:368:12: warning: variable 'caesar_x_variable_0' set but not used [-Wunused-but-set-variable]
  368 | ADT_STRING caesar_x_variable_0 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_19855_1.c:465:12: warning: variable 'caesar_x_variable_2' set but not used [-Wunused-but-set-variable]
  465 | ADT_STRING caesar_x_variable_2 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~


###  1.2 Product Loyalty query 

In [104]:
query_path = "../queries/mcl/product_loyalty.mcl"
print_query_content(query_path)

(* Product Loyalty :*)

macro QUERY () =
     {!"G"}.
     {SELF ?C0:String ?P0:String !"T0"}.
     {?any}*.
     {SELF ?C1:String ?P1:String !"T1" where
        (inter (string2natset (P1), string2natset (P0)) <> empty)
     }
end_macro

macro FINAL_STATE () =
    [ not { SELF ... } ] false
end_macro

<
    (true* . QUERY . true*)+
> FINAL_STATE


In [105]:
%%bash -s $file_path $query_path
$CADP/com/bcg_open $1 evaluator4 -diag $2

bcg_open: using ``/home/abdelouahab/cadp//bin.x64/evaluator4.a''
bcg_open: running ``evaluator4 -diag ../queries/mcl/product_loyalty.mcl'' for ``../graph/bcg/sample.bcg''

TRUE
*** diagnostic sequence found at depth 5

<initial state>
"G"
"SELF !:1:2:3:4:5:6: !:1:3:4: !T0"
"G"
"SELF !:1:2:3:4:5:6:9: !:1:3:15 !T1"
"ST"
<goal state>


/tmp/caesar_21249_1.c: In function 'caesar_iterate_boolean_variable_0':
/tmp/caesar_21249_1.c:368:12: warning: variable 'caesar_x_variable_0' set but not used [-Wunused-but-set-variable]
  368 | ADT_STRING caesar_x_variable_0 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_21249_1.c:465:12: warning: variable 'caesar_x_variable_2' set but not used [-Wunused-but-set-variable]
  465 | ADT_STRING caesar_x_variable_2 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~


### 1.3 Promotion only

In [113]:
query_path = "../queries/mcl/promotion_only.mcl"
print_query_content(query_path)

(* Promotion only  :*)

macro QUERY () =
     {!"G"}.
     {SELF ?C0:String ?P0:String !"T0"}.
     {?any}*.
     {SELF ?C1:String ?P1:String !"T1"}.
     {?any}*.
     {SELF ?C2:String ?P2:String !"T2" where
             (diff (string2natset (P2), union(string2natset (P0),string2natset (P1))) <> empty)
     }
end_macro

macro FINAL_STATE () =
    [ not { SELF ... } ] false
end_macro

<
    (true* . QUERY . true*)+
> FINAL_STATE


In [14]:
%%bash -s $file_path $query_path
$CADP/com/bcg_open $1 evaluator4 -diag $2

bcg_open: using ``/home/abdelouahab/cadp//bin.x64/evaluator4.a''
bcg_open: running ``evaluator4 -diag ../queries/mcl/product_adoption.mcl'' for ``../graph/bcg/sample.bcg''

TRUE
*** diagnostic sequence found at depth 5

<initial state>
"G"
"SELF !:1:2:3:4:5:6: !:1:3:4: !T0"
"G"
"SELF !:1:2:3:4:5:6:9: !:1:3:15 !T1"
"ST"
<goal state>


/tmp/caesar_21852_1.c: In function 'caesar_iterate_boolean_variable_0':
/tmp/caesar_21852_1.c:368:12: warning: variable 'caesar_x_variable_0' set but not used [-Wunused-but-set-variable]
  368 | ADT_STRING caesar_x_variable_0 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~
/tmp/caesar_21852_1.c:465:12: warning: variable 'caesar_x_variable_2' set but not used [-Wunused-but-set-variable]
  465 | ADT_STRING caesar_x_variable_2 = NULL;
      |            ^~~~~~~~~~~~~~~~~~~


## 2. SVL queries

In [ ]:
# Choose target graph 
file_name = "sample.bcg"
file_path = f"../graph/bcg/{file_name}"

In [16]:
before_promo_period = "2018-09-01"
promo_period ="2018-12-01"
after_promo_period = ":2019-02-01"

### 2.1 Adoption query

In [66]:
query_name = "product_adoption.svl"
query_path = f"../queries/svl/{query_name}"
file_path = f'../graph/bcg/9M-5-[1-2001]-[a]-lcm'

In [67]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_adoption.svl $1 ":3461025:7461024:" ":2018-09-01" ":2018-12-01" ":2019-02-01"


property PRODUCT_ADOPTION
 | Product adoption

TRUE

[Customers] [Promotional products] Period

[211249] [7461024] :2018-12-01
[264552] [3461025] :2018-12-01
[277744] [3461025] :2018-12-01
[322520, 526900] [3461025] :2018-12-01
[322520, 575459] [3461025] :2018-12-01
[336921, 575459] [3461025] :2018-12-01
[400827, 526900, 575459] [3461025] :2018-12-01
[400827, 575459, 1044697] [3461025] :2018-12-01
[400827, 575459] [3461025] :2018-12-01
[526900, 575459] [3461025] :2018-12-01
[526900] [3461025] :2018-12-01
[575459, 1044697] [3461025] :2018-12-01
[575459, 819320] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01
[575459] [7461024] :2018-12-01
[819320] [3461025] :2018-12-01



###  2.2 Loyalty query 

In [88]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_loyalty.svl $1 ":3461025:7461024:" ":2018-12-01" ":2019-02-01"


property product_loyalty

TRUE

[Customers] [Promotional products] Period

[1044697] [3461025] :2018-12-01
[211249] [7461024] :2018-12-01
[575459, 1044697, 1163180] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01



### 2.3 Pomotion only query

In [108]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl promotion_only.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property promotion_only

TRUE

[Customers] [Promotional products] Period




In [109]:
!gedit promotion_only.svl
